# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 3-1. Задача TSP: инкрементальные алгоритмы.

В этой задаче Вам предлагается сравнить алгоритмы Nearest Neighbour и Nearest Insertion в задаче Euclidean TSP.

**Даны:**
* Координаты точек плоскости, являющихся вершинами графа.

**Найти:**
* Перестановку вершин, задающих минимальный по длине гамильтонов цикл в графе.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP.
* Реализуйте функции `solve_tsp_nearest_neighbour` и `solve_tsp_nearest_insertion`.
* Запустите функцию `run_all()`, чтобы протестировать свой код и сравнить качество решений, получаемых Nearest Neighbour и Nearest Insertion. Сильно ли они отличаются? Запишите свои качественные выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [12]:
import time
import os
import math

import numpy as np

In [7]:
def read_tsp_instance(path: str) -> list:
    with open(path, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0,0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates

def euclidean_distance(point1: tuple, point2: tuple) -> float:
    if len(point1) != len(point2):
        raise ValueError("Points must have equal dimensions")
    
    distance = 0
    for j in range(len(point1)):
        distance += (point1[j] - point2[j]) ** 2
    return math.sqrt(distance)

def calculate_tour_length(instance, permutation):
    assert(len(instance) == len(permutation))
    
    n = len(permutation)
    return sum(euclidean_distance(instance[permutation[i]], instance[permutation[(i+1) % n]]) for i in range(len(permutation)))

In [18]:
# input -- [(x1, y1), (x2, y2)]
# output -- nearest neighbour algo vertices sequence
def solve_tsp_nearest_neighbour(instance: list):
    n = len(instance)
    def w(i, j):
        return euclidean_distance(instance[i], instance[j])
    
    not_visited = np.array(range(1, n))
    answer = np.array([0])
    
    for i in range(1, n):
        distances = [w(answer[-1], not_visited[j]) for j in range(n - i)]
        min_i = np.argmin(distances)
        answer = np.insert(answer, i, not_visited[min_i])
        not_visited = np.delete(not_visited, min_i)
    
    return answer

In [9]:
def solve_tsp_nearest_insertion(instance):
    n = len(instance)
    if n == 1:
        return [0]
    
    # cycle format: [v1, v2, ...., v_n, v1]
    # returns (nearest vertex index, insertion index)
    def closest_to_cycle(vertex_indices: list, cycle: list):
        min_dist = math.inf
        closest_i = -1
        insertion_i = -1
        
        for edge_i, v1_i in enumerate(cycle[:-1]):
            v2_i = cycle[edge_i + 1] 
            edge_w = euclidean_distance(instance[v1_i], instance[v2_i])
            
            for vertex_i in vertex_indices:
                cur_dist = euclidean_distance(instance[v1_i], instance[vertex_i]) + \
                           euclidean_distance(instance[v2_i], instance[vertex_i]) - edge_w
                
                if cur_dist < min_dist:
                    min_dist = cur_dist
                    closest_i = vertex_i
                    insertion_i = edge_i
        
        return closest_i, insertion_i

    v0_i = 0
    not_visited = list(range(1, n))
    v1_i = min(not_visited, key=lambda i: euclidean_distance(instance[i], instance[v0_i]))
    not_visited.remove(v1_i)
    
    cur_cycle = [v0_i, v1_i, v0_i]
    
    for iteration_num in range(len(instance) - 2):
        nearest_vertex_i, insertion_i = closest_to_cycle(not_visited, cur_cycle)
        cur_cycle = cur_cycle[:insertion_i + 1] + [nearest_vertex_i] + cur_cycle[insertion_i + 1:]
        not_visited.remove(nearest_vertex_i)
    
    assert(len(cur_cycle) == len(instance) + 1)
    return cur_cycle[:-1]

In [10]:
def run_all():
    instance_filenames = ['d198.tsp']#, 'd493.tsp', 'd657.tsp', 'd2103.tsp', 'pr107.tsp', 'pr152.tsp', 'pr439.tsp']
    for filename in instance_filenames:
        path = 'tsp-instances/{file}'.format(file=filename)
        if not os.path.exists(path):
            print('File not found: “{}”. Skipping this instance.'.format(path))
            continue
        instance = read_tsp_instance(path)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality_nn = calculate_tour_length(instance, solve_tsp_nearest_neighbour(instance))
        time_nn = time.monotonic()-time_start
        time_start = time.monotonic()
        quality_ni = calculate_tour_length(instance, solve_tsp_nearest_insertion(instance))
        time_ni = time.monotonic()-time_start
        print(' done \n NN: {:.2} seconds, Tour length {} \n NI: {:.2} seconds, Tour length {}'.format(time_nn, int(quality_nn), time_ni, int(quality_ni)))

In [19]:
run_all()

Solving instance d198.tsp… done 
 NN: 0.053 seconds, Tour length 18620 
 NI: 2.8 seconds, Tour length 17631


In [94]:
run_all()

Solving instance d198.tsp… done 
 NN: 0.042 seconds, Tour length 18830 
 NI: 2.8 seconds, Tour length 17631
Solving instance d493.tsp… done 
 NN: 0.14 seconds, Tour length 44160 
 NI: 4.2e+01 seconds, Tour length 39982
Solving instance d657.tsp… done 
 NN: 0.25 seconds, Tour length 62860 
 NI: 1.1e+02 seconds, Tour length 57906
Solving instance d2103.tsp… done 
 NN: 2.7 seconds, Tour length 92247 
 NI: 3.6e+03 seconds, Tour length 87665
Solving instance pr107.tsp… done 
 NN: 0.0073 seconds, Tour length 47464 
 NI: 0.48 seconds, Tour length 52587
Solving instance pr152.tsp… done 
 NN: 0.015 seconds, Tour length 85314 
 NI: 1.3 seconds, Tour length 87848
Solving instance pr439.tsp… done 
 NN: 0.12 seconds, Tour length 131702 
 NI: 3.2e+01 seconds, Tour length 130254


## Выводы
Как видно по результатам запусков, нельзя заранее сказать, какой алгоритм сработает лучше, но при том